In [4]:
!pip install torch

     -------------------------------------- 172.4/172.4 MB 5.0 MB/s eta 0:00:00


In [13]:
da1 = pd.read_csv("/Users/dongh/competition DATA/train_data.csv", encoding='cp1252')

print(da1)

                 regdate  temp  humi    co2  dust_pm1  dust_pm_25  dust_pm_10  \
0        2023-01-01 0:00  21.2  34.0  414.0      14.0        14.0        14.0   
1        2023-01-01 0:01  21.2  34.0  411.0      13.0        13.0        13.0   
2        2023-01-01 0:02  21.2  34.0  409.0      14.0        14.0        14.0   
3        2023-01-01 0:03  21.2  34.0  409.0      14.0        14.0        14.0   
4        2023-01-01 0:04  21.2  34.0  415.0      14.0        14.0        14.0   
...                  ...   ...   ...    ...       ...         ...         ...   
388795  2023-01-27 23:55  17.4  17.0  414.0       9.0         9.0         9.0   
388796  2023-01-27 23:56  17.4  17.0  410.0       9.0         9.0         9.0   
388797  2023-01-27 23:57  17.4  17.0  417.0       9.0         9.0         9.0   
388798  2023-01-27 23:58  17.5  17.0  414.0       9.0         9.0         9.0   
388799  2023-01-27 23:59  17.4  17.0  415.0       9.0         9.0         9.0   

        illuminance     voc

In [15]:
da = pd.read_csv('/Users/dongh/competition DATA/train_data.csv', encoding='cp1252')

data = da.rename(columns={'Àç½ÇÀÎ¿ø': 'person'})

# 이상치 식별 및 제거
for column in data.columns[1:]:
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    lower_fence = q1 - 1.5 * iqr
    upper_fence = q3 + 1.5 * iqr

    outliers = (data[column] < lower_fence) | (data[column] > upper_fence)
    data = data[~outliers]

# 인덱스 재설정
data = data.reset_index(drop=True)

# 수정된 데이터 저장
save_path = '/Users/dongh/competition DATA/modified_train_data.csv'
data.to_csv(save_path, index=False)

print(data)

df_test = pd.read_csv("/Users/dongh/competition DATA/test_data.csv", encoding='ISO-8859-1')

data1 = df_test.rename(columns={'Àç½ÇÀÎ¿ø': 'person'})
# 수정된 데이터 저장
save_path = '/Users/dongh/competition DATA/modified_test_data.csv'
data1.to_csv(save_path, index=False)
print(data1)

                 regdate  temp  humi    co2  dust_pm1  dust_pm_25  dust_pm_10  \
0       2023-01-01 11:03  20.1  31.0  570.0      20.0        20.0        20.0   
1       2023-01-01 11:06  20.1  31.0  577.0      20.0        20.0        20.0   
2       2023-01-01 11:07  20.1  31.0  577.0      20.0        20.0        20.0   
3       2023-01-01 11:09  20.1  31.0  570.0      20.0        20.0        20.0   
4       2023-01-01 11:11  20.1  31.0  567.0      20.0        20.0        20.0   
...                  ...   ...   ...    ...       ...         ...         ...   
279973  2023-01-27 23:55  17.4  17.0  414.0       9.0         9.0         9.0   
279974  2023-01-27 23:56  17.4  17.0  410.0       9.0         9.0         9.0   
279975  2023-01-27 23:57  17.4  17.0  417.0       9.0         9.0         9.0   
279976  2023-01-27 23:58  17.5  17.0  414.0       9.0         9.0         9.0   
279977  2023-01-27 23:59  17.4  17.0  415.0       9.0         9.0         9.0   

        illuminance  voc   

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import re
import csv
import gensim
from sklearn.preprocessing import normalize
from itertools import combinations
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim


def pre_df(df):
    df = df.dropna()
    label = df['person']
    df = df.drop('person', axis=1)
    df = df.drop('regdate', axis=1)
    df = df.drop('PIR', axis=1)
    df_33 = df.values.reshape(-1,3,3)
    return df_33, label
#0~255 스케일링 함수
def scale_255(array,i,j):
    # i,j 위치의 요소들을 추출하여 배열 생성
    first_elements = array[:, i, j]
    # 첫 번째 요소들의 최솟값과 최댓값 계산
    min_value = np.min(first_elements)
    max_value = np.max(first_elements)
    # 스케일링 수행
    scaled_array = (first_elements - min_value) * (255 / (max_value - min_value))
    return scaled_array
def make_28(df_33):
    # 결과 행렬을 저장할 빈 배열 생성
    expanded_data = np.zeros((len(df_33), 28, 28))
    # df_33의 각 3x3 배열에 대해 반복
    for idx, arr in enumerate(df_33):
        expanded_arr = np.repeat(arr,9,axis=0)
        expanded_arr = np.repeat(expanded_arr,9,axis=1)
        # 패딩을 추가한 새로운 28x28 배열 생성
        expanded_arr = np.pad(expanded_arr, ((0, 1), (0, 1)), mode='constant')
        expanded_data[idx] = expanded_arr
    return expanded_data

df = pd.read_csv("/Users/dongh/competition DATA/modified_train_data.csv", encoding='ISO-8859-1')
df_33, label = pre_df(df)
#스케일링 함수 전체 적용
for i in range(3):
    for j in range(3):
        df_33[:,i,j] = scale_255(df_33,i,j)
df_33 = make_28(df_33)
# 배열을 PyTorch Tensor로 변환
input_data = torch.from_numpy(df_33).unsqueeze(1).float()  # 차원 수정
# 라벨 데이터를 PyTorch Tensor로 변환
target = torch.tensor(label.values)
print(input_data.shape)
#test
df_test = pd.read_csv("/Users/dongh/competition DATA/modified_test_data.csv", encoding='ISO-8859-1')
df_test_33, label_test = pre_df(df_test)
#스케일링 함수 전체 적용
for i in range(3):
    for j in range(3):
        df_test_33[:,i,j] = scale_255(df_test_33,i,j)
df_test_33 = make_28(df_test_33)
# 배열을 PyTorch Tensor로 변환
input_test = torch.from_numpy(df_test_33).unsqueeze(1).float()  # 차원 수정
# 라벨 데이터를 PyTorch Tensor로 변환
target_test = torch.tensor(label_test.values)
print(input_test.shape)

# LeNet-5 모델 정의 (시퀀셜)
model = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 400),
    nn.ReLU(),
    nn.Linear(400, 120),
    nn.ReLU(),
    nn.Linear(120, 7)
)

# 모델 학습 또는 예측 수행

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 학습
num_epochs = 10
for epoch in range(num_epochs):
    # 입력 데이터의 forward pass
    outputs = model(input_data)

    # 손실 계산
    loss = criterion(outputs, target)

    # 역전파 및 가중치 업데이트
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 예측
    with torch.no_grad():
        outputs = model(input_test)
        _, predicted = torch.max(outputs.data, 1)

    # 예측 결과와 실제 라벨 비교하여 정확도 계산
    accuracy = (predicted == target_test).sum().item() / target_test.size(0)

    # 현재 에폭의 손실과 정확도 출력
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, Accuracy: {accuracy}')


torch.Size([201920, 1, 28, 28])
torch.Size([84920, 1, 28, 28])
Epoch [1/10], Loss: 2.2042932510375977, Accuracy: 0.3996231747527084
Epoch [2/10], Loss: 1.6127078533172607, Accuracy: 0.45370937352802637
Epoch [3/10], Loss: 1.5080968141555786, Accuracy: 0.4652614225153085
Epoch [4/10], Loss: 1.4698766469955444, Accuracy: 0.47204427696655676
Epoch [5/10], Loss: 1.4455180168151855, Accuracy: 0.4738577484691474
Epoch [6/10], Loss: 1.426044225692749, Accuracy: 0.4744112105511069
Epoch [7/10], Loss: 1.4092475175857544, Accuracy: 0.4771902967498822
Epoch [8/10], Loss: 1.3943136930465698, Accuracy: 0.4773551577955723
Epoch [9/10], Loss: 1.3810144662857056, Accuracy: 0.47740226095148375
Epoch [10/10], Loss: 1.3692582845687866, Accuracy: 0.47747291568535094
